In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd /home/dfried/projects/ImageCaptioning.pytorch

/home/dfried/projects/ImageCaptioning.pytorch


In [3]:
import sys

In [4]:
sys.path.append("/home/dfried/projects/ImageCaptioning.pytorch")

In [5]:
import numpy as np
import torch
import einops

In [6]:
import tqdm

In [7]:
import pickle

In [8]:
import pprint

In [9]:
import argparse
import captioning.utils.opts as opts
import captioning.utils.misc as utils
import captioning.models as models
from captioning.utils import eval_utils
from captioning import modules

In [10]:
from captioning.data.dataloader import DataLoader
from captioning.data.dataloaderraw import DataLoaderRaw

In [11]:
parser = argparse.ArgumentParser()
opts.add_eval_options(parser)
opts.add_diversity_opts(parser)

In [12]:
opt = parser.parse_args([])

In [13]:
opt

Namespace(batch_size=0, beam_size=1, block_trigrams=0, coco_json='', decoding_constraint=0, diversity_lambda=0.5, dump_images=1, dump_json=1, dump_path=0, eval_oracle=1, group_size=1, id='', image_folder='', image_root='', input_att_dir='', input_box_dir='', input_fc_dir='', input_json='', input_label_h5='', language_eval=0, length_penalty='', max_length=20, num_images=-1, remove_bad_endings=0, sample_method='greedy', sample_n=1, sample_n_method='sample', split='test', suppress_UNK=1, temperature=1.0, verbose=1, verbose_beam=1, verbose_captions=0, verbose_loss=0)

In [37]:
# model_name = 'updown_contrastive_distr-10_dct-random_cbs_mlp_soft-em'
# model_name = 'updown_contrastive_distr-10_dct-closest+random_cbs_mlp_soft-em'
# model_name = 'updown_contrastive_distr-10_cbs_mlp_soft-em'
# model_name = 'updown_contrastive_cbs'
# model_name = 'updown_contrastive_distr-10_alpha-5.0_resume-em'
# model_name = 'updown_contrastive_distr-10_finetune-em'
model_name = 'updown_contrastive_distr-10_dct-closest+random_alpha-1.0_resume-em'
# model_name = 'updown_contrastive_dct-random_cbs_mlp_soft-em'
model_fname = f'log_{model_name}/model-best.pth'
infos_fname = f'log_{model_name}/infos_{model_name}.pkl'

In [38]:
with open(infos_fname, 'rb') as f:
    infos = utils.pickle_load(f)
infos.keys()

dict_keys(['iter', 'epoch', 'loader_state_dict', 'vocab', 'opt', 'best_val_score'])

In [39]:
replace = ['input_fc_dir', 'input_att_dir', 'input_box_dir', 'input_label_h5', 'input_json', 'batch_size', 'id']
ignore = ['start_from']

for k in vars(infos['opt']).keys():
    if k in replace:
        setattr(opt, k, getattr(opt, k) or getattr(infos['opt'], k, ''))
    elif k not in ignore:
        if not k in vars(opt):
            vars(opt).update({k: vars(infos['opt'])[k]}) # copy over options from model

In [40]:
opt.vocab = infos['vocab']

In [41]:
pprint.pprint(vars(infos['opt']))

{'att_feat_size': 2048,
 'att_hid_size': 512,
 'batch_size': 10,
 'beam_size': 1,
 'bleu_reward_weight': 0,
 'block_trigrams': 0,
 'cached_tokens': 'coco-train-idxs',
 'cached_transformer': False,
 'caption_model': 'updown',
 'cfg': 'configs/updown/updown_contrastive.yml',
 'checkpoint_path': './log_updown_contrastive_distr-10_dct-closest+random_alpha-1.0_resume-em',
 'cider_reward_weight': 1,
 'contrastive_after': 0,
 'contrastive_em': 'soft',
 'current_lr': 0.0003200000000000001,
 'data_in_memory': False,
 'decoding_constraint': 0,
 'diversity_lambda': 0.5,
 'drop_prob_lm': 0.5,
 'entropy_reward_weight': 0,
 'eval_oracle': 1,
 'fc_feat_size': 2048,
 'grad_clip_mode': 'value',
 'grad_clip_value': 0.1,
 'group_size': 1,
 'id': 'updown_contrastive_distr-10_dct-closest+random_alpha-1.0_resume-em',
 'index_serialization_root_path': 'data/cocobu_indices',
 'input_att_dir': 'data/cocobu_att',
 'input_box_dir': 'data/cocotalk_box',
 'input_encoding_size': 1000,
 'input_fc_dir': 'data/cocotal

In [42]:
model = models.setup(opt)

In [43]:
del opt.vocab

In [44]:
CUDA = True

In [45]:
model.load_state_dict(torch.load(model_fname, map_location='cpu'), strict=False)
if CUDA:
    model.cuda()
model.eval()

UpDownModel(
  (embed): Sequential(
    (0): Embedding(9488, 1000)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (fc_embed): Sequential(
    (0): Linear(in_features=2048, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (att_embed): Sequential(
    (0): Linear(in_features=2048, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (logit): Linear(in_features=1000, out_features=9488, bias=True)
  (ctx2att): Linear(in_features=1000, out_features=512, bias=True)
  (distractor_scorer): DistractorScorer(
    (scorer): Sequential(
      (0): Linear(in_features=4096, out_features=200, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=200, out_features=1, bias=True)
    )
  )
  (core): UpDownCore(
    (att_lstm): LSTMCell(3000, 1000)
    (lang_lstm): LSTMCell(2000, 1000)
    (attention): Attention(
      (h2att): Linear(in_features=1000, out_features=5

In [46]:
opt.num_workers = 0

In [47]:
loader = DataLoader(opt, shuffle_override=False, wrap_override=False, build_nearest_neighbor_indices_for_splits=['train'], index_serialization_root_path='data/cocobu_indices/')

DataLoader loading json file:  data/cocotalk.json
vocab size is  9487
DataLoader loading h5 file:  data/cocotalk_fc data/cocobu_att data/cocotalk_box data/cocotalk_label.h5
max sequence length in data is 16
read 123287 image features
assigned 113287 images to split train
assigned 5000 images to split val
assigned 5000 images to split test


In [48]:
vocab = loader.get_vocab()

In [49]:
PAD_ID = 0

In [50]:
batch = loader.get_batch('val')

In [51]:
def get_captions_from_batch(data):
    batch_captions = []
    for batch_labels in data['labels']:
        instance_captions = []
        for img_labels in batch_labels:
            neighbor_captions = []
            for neighbor_labels in img_labels:
                caption = [vocab[str(ix.item())] for ix in neighbor_labels if ix != 0]
                neighbor_captions.append(caption)
            instance_captions.append(neighbor_captions)
        batch_captions.append(instance_captions)
    return batch_captions

In [52]:
def wrap_tag(tag, inner):
    return f'<{tag}>{inner}</{tag}>'

def image_html(image_path, width=300, border=False):
    if border:
        style = ' style="border: 5px solid #0FF" '
    else:
        style = ''
    return f'<img width={width} src="{image_path}" {style}></img>'

def captions_html(captions):
    #return wrap_tag('p', '<br>'.join(' '.join(cap) for cap in captions))
    return wrap_tag('ol', ''.join(wrap_tag('li', cap) for cap in captions))

def images_html(image_paths, width=300, num_per_row=5, target=None, captions=None, titles=None):
    rows = []
    for ix in range(0, len(image_paths), num_per_row):
        items = [wrap_tag('td', image_html(image_paths[image_ix], width=width, border=image_ix == target)) 
                 for image_ix in range(ix, ix+num_per_row) if image_ix < len(image_paths)]
        rows.append(wrap_tag('tr', ''.join(items)))
        if titles is not None:
            title_html = [
                wrap_tag('td', wrap_tag('b', titles[image_ix]))
                for image_ix in range(ix, ix+num_per_row)
                if image_ix < len(image_paths)
            ]
            rows.append(wrap_tag('tr', ''.join(title_html)))
        if captions is not None:
            cap_html = [
                wrap_tag('td', captions_html(captions[image_ix]))
                for image_ix in range(ix, ix+num_per_row)
                if image_ix < len(image_paths)
            ]
            rows.append(wrap_tag('tr', ''.join(cap_html)))
    return wrap_tag('table', ''.join(rows))

def display_images(image_paths, width=300, num_per_row=5, target=None, captions=None, titles=None):
    display(HTML(images_html(image_paths, width=width, num_per_row=num_per_row, target=target, captions=captions, titles=titles)))

In [53]:
def get_neighbor_batch(img_fc_feat, k, include_self=False, self_ix=None):
    assert len(img_fc_feat.shape) == 1, "should be the features for a single image"
    D, I = index.search(img_fc_feat[None], k)
    n_images, k_ = D.shape
    assert n_images == 1
    indices = []
    if include_self:
        assert self_ix is not None
        indices.append(self_ix)
    indices.extend([ixs[i] for i in I.flatten()])
    data = [loader.dataset[ix, 0, False] for ix in indices]
    batch = loader.dataset.collate_func(data, 'train')
    return batch

In [54]:
def display_neighbors(features, k=5, num_per_row=5):
    index = loader.indices['train']
    neighbor_batch = index.get_neighbor_batch(loader, features, k)
    paths_k = [d['file_path'] for d in neighbor_batch['infos']]
    captions_k = [[' '.join(c) for c in ns] for cs in get_captions_from_batch(neighbor_batch) for ns in cs]
    display_images(paths_k, captions=captions_k, num_per_row=num_per_row)

In [55]:
pdb on

Automatic pdb calling has been turned ON


In [36]:
display_neighbors(batch['fc_feats'].numpy(), k=8, num_per_row=4)

,,,
bulls dogs and people all share the same streeta black and white cow standing in a markettwo cows and two small dogs hang out in front a street markettwo cows and two dogs and people at an open air markettwo dogs and two cows are amongst people in a market,two people standing next to two huge elephantstwo elephants standing near two people with mountains backgroundtwo elephants in the foreground and people in a dirt field3 saddled elephants and people in a dirt fieldthere are two people standing near two elephants,a herd of bulls walking through a town guided by mena herd of cattle is pushed through a street past people and vendorscows walking in a row down an indian streeta herd of cattle being led down a dirt roada bunch of cows are standing in a pin,a huge crowd of people gathered around small tents and livestocka very large festival in a rural country with man leading cattlea crowd of people standing around a herd of cowsa man pulling two cows by ropes with a lot of people gathered togetherUNK and animals standing around a campground near a city
,,,
people walking sheep around a grassy area with people watchinga group of sheep inside of a fenced areapeople are walking inside a gate with many large sheep near an audiencea crowd of people standing around a field with sheeppeople walking UNK on leashes around in a fenced in area,a woman holding a harness on a cow in a festivala cow a horse and a sheep with some peoplea person teaching a cow some tricks in a parksome people that are showing off their animals for a judge to seea couple of people on a grassy area with different animals,this is a girl feeding an elephant some vegetablesan elephants trunk taking something from a womans handa young girl feeding an elephant bananas at an animal sanctuarysome people are feeding some elephants dirt water and treesadults interacting with native elephants near river on sunny day,a woman tending to a brown cow with a rope wrapped around its mouthan adult cow and a baby cow have harnesses on their mouth near a womana woman that is standing next to a cowa woman petting a cow that is tied upa close up of a woman petting a cow that is tied up
,,,
a kitchen with a sink a shelf and a chairan all white kitchen with a sink and stovea kitchen with a shelf cabinets a sink and stovea home kitchen with a door leading to the living rooma narrow kitchen with beams in the ceiling leads to a family room,a toddler in a kitchen trying to use a vacuum cleanera mom and a kid in a green kitchena toddler standing around in a kitchen with his mom at the doora small boy standing in a kitchen next to a countera mother and baby in the kitchen next to a cabinet and oven,a woman standing next to a kitchen sinkw woman is at the sink in a clean kitchena woman is standing in a kitchen next to the sinkbright kitchen with woman doing something at the sinka person in a very big kitchen by the sink,a kitchen filled with appliances and dishes on countersa somewhat disorganized looking kitchen with old wooden flooringsome of the cabinets in the kitchen were left opena kitchen has white cabinets and a wood flooran l shaped white kitchen with green marble counter tops gets plenty of natural light from
,,,
a refrigerator that has a microwave above it in the kitchena kitchen with wood cabinets white refrigerator white stove and a microwave above the refrigeratormicrowave sitting inside a wall with a refrigeratorthe kitchen includes a microwave on a refrigeratorthis is the kitchen of a smaller apartment,a kitchen with cabinets and sink on the left and fridge on the righta kitchen with a large sliding glass door providing a view to a backyarda kitchen with a sink oven stove and a refrigeratorthis is a photo of a large kitchen and the door leading in is openan empty kitchen with a sliding glass door open,a black and white kitchen with an island table areaa kitchen with a bunch of shelfs in ita kitchen that has a black counter top and white cabinetsthis kitchen has all white ca

In [57]:
def maybe_cuda(tensor):
    if tensor is None:
        return None
    if CUDA:
        return tensor.cuda()
    return tensor

In [56]:
def display_em_model(batch, k=5, num_per_row=5, neighbor_types=['closest'], choose_distractor=False, beam_size=5, distractor_selection_method='max'):
    index = loader.indices['train']
    opt_new = vars(opt)
    for neighbor_type in neighbor_types:
        assert neighbor_type in ['closest', 'random']
    opt_new['pragmatic_distractor_candidate_types'] = neighbor_types
    opt_new['pragmatic_distractors'] = k
    opt_new['beam_size'] = beam_size
    if choose_distractor:
        opt_new['pragmatic_distractor_type'] = 'choose_within_closest'
    else:
        # misnomer; this actually means use all the available distractors
        opt_new['pragmatic_distractor_type'] = 'closest'
    batch_size = len(batch['infos'])
    with torch.no_grad():
        fc_feats, att_feats, att_masks = maybe_cuda(batch['fc_feats']), maybe_cuda(batch['att_feats']), maybe_cuda(batch['att_masks'])
        model._sample_beam(fc_feats, att_feats, att_masks, opt=opt_new)
        beam_captions = [[utils.decode_sequence(model.vocab, _['seq'].unsqueeze(0))[0] for _ in model.done_beams[ix]] for ix in range(batch_size)]
        model._sample_contrastive_beam(
            fc_feats, att_feats, att_masks,
            opt_new,  data=batch, loader=loader, nearest_neighbor_index=index, 
            distractor_selection_method=distractor_selection_method,
        )
        contrastive_beam_captions = [[utils.decode_sequence(model.vocab, _['seq'].unsqueeze(0))[0] for _ in model.done_beams[ix]] for ix in range(batch_size)]
    for ix in range(len(batch['infos'])):
        path = batch['infos'][ix]['file_path']
        display_images([path, path], captions=[beam_captions[ix], contrastive_beam_captions[ix]], titles=['normal', 'contrastive'])
        
        neighbor_paths = [d['file_path'] for d in model.neighbor_infos[ix]]
        if hasattr(model, 'dlp'):
            scores = model.dlp[ix].exp().detach().cpu().numpy()
            scores, neighbor_paths = zip(*sorted(zip(scores, neighbor_paths), reverse=True))
            score_strings = ['{:.4f}'.format(x) for x in scores]
        else:
            score_strings = None
        if choose_distractor:
            display_images([d['file_path'] for d in model.distractor_infos[ix]], num_per_row=2)
        display_images(neighbor_paths, titles=score_strings, num_per_row=5)
        print()
#     paths_k = [d['file_path'] for d in neighbor_batch['infos']]
#     captions_k = [[' '.join(c) for c in ns] for cs in get_captions_from_batch(neighbor_batch) for ns in cs]
#     display_images(paths_k, captions=captions_k, num_per_row=num_per_row)

In [60]:
# rinetuned-resume 1.0 with closest+random
display_em_model(batch, k=20, neighbor_types=['closest', 'random'], choose_distractor=True)

,
normal,contrastive
a little boy holding a sheep in a fielda young boy holding a sheep in a fielda little boy is holding a sheep in the grassa little boy is holding a sheep in a fielda little girl is holding a sheep in the grass,a little boy holding a sheep in a fielda young boy holding a sheep in a fielda little boy is holding a sheep in the grassa young child is holding a sheep in a fielda little boy is holding a sheep in a field


,


,,,,
0.2691,0.1931,0.1559,0.0880,0.0657
,,,,
0.0427,0.0331,0.0267,0.0257,0.0241
,,,,
0.0226,0.0212,0.0115,0.0034,0.0026
,,,,
0.0024,0.0024,0.0014,0.0011,0.0010
,,,,
0.0008,0.0007,0.0007,0.0006,0.0004
,,,,


,
normal,contrastive
a kitchen with white cabinets and white appliancesa kitchen with white appliances and white cabinetsa kitchen with white cabinets and white cabinetsa kitchen with white appliances and white appliancesa small kitchen with white appliances and white cabinets,a kitchen with white cabinets and white appliancesa kitchen with white appliances and white cabinetsa kitchen with white cabinets and white cabinetsa kitchen with white appliances and white appliancesa small kitchen with white appliances and white cabinets


,


...

,


,,,,
0.1721,0.1655,0.1328,0.1149,0.1096
,,,,
0.0863,0.0675,0.0607,0.0347,0.0229
,,,,
0.0084,0.0080,0.0072,0.0048,0.0035
,,,,
0.0009,0.0000,0.0000,0.0000,0.0000
,,,,
0.0000,0.0000,0.0000,0.0000,0.0000
,,,,


,
normal,contrastive
a yellow frisbee sitting on top of a yellow fire hydranta yellow fire hydrant sitting on top of a dirt fielda yellow fire hydrant sitting on top of a dirt roada yellow fire hydrant sitting on top of a patch of dirta yellow fire hydrant sitting on top of a patch of grass,a yellow fire hydrant with a banana on ita yellow fire hydrant sitting on top of a dirt fielda yellow fire hydrant with a yellow topa yellow fire hydrant sitting on top of a dirt roada yellow fire hydrant sitting on top of a patch of dirt


,


,,,,
0.2883,0.1074,0.0800,0.0786,0.0785
,,,,
0.0662,0.0607,0.0564,0.0397,0.0389
,,,,
0.0268,0.0265,0.0148,0.0136,0.0119
,,,,
0.0092,0.0008,0.0005,0.0004,0.0003
,,,,
0.0002,0.0002,0.0001,0.0001,0.0000
,,,,


In [62]:
# rinetuned-resume 1.0 with closest+random
display_em_model(next(loader.iters['val']), k=20, neighbor_types=['closest', 'random'], choose_distractor=True)

,
normal,contrastive
a blue truck is parked in the grassa blue truck is parked in a fielda blue truck is parked in a grassy fieldan old blue truck is parked in the grassan old blue truck is parked in a field,a blue truck is parked in the grassan old blue truck is parked in the grassa blue truck is parked in a fieldan old blue truck is parked in a fieldan old blue truck is parked in a grassy field


,


,,,,
0.1426,0.1283,0.1262,0.1043,0.1025
,,,,
0.0696,0.0686,0.0638,0.0621,0.0536
,,,,
0.0389,0.0302,0.0035,0.0024,0.0016
,,,,
0.0003,0.0001,0.0001,0.0001,0.0001
,,,,
0.0001,0.0001,0.0001,0.0001,0.0001
,,,,


,
normal,contrastive
a black cat drinking water from a toileta black cat drinking water out of a toileta cat sitting on top of a toilet seata cat sitting on a toilet seat in a bathrooma cat sitting on top of a toilet in a bathroom,a black cat drinking water from a toileta black cat drinking water out of a toileta cat sitting on top of a toilet seata cat sitting on a toilet seat in a bathrooma cat sitting on top of a toilet in a bathroom


,


...

,


,,,,
0.1341,0.0900,0.0779,0.0697,0.0688
,,,,
0.0654,0.0632,0.0630,0.0626,0.0574
,,,,
0.0539,0.0522,0.0438,0.0332,0.0223
,,,,
0.0144,0.0125,0.0059,0.0035,0.0025
,,,,
0.0011,0.0007,0.0003,0.0003,0.0003
,,,,


,
normal,contrastive
a green traffic light sitting on the side of a roada green traffic light sitting next to a traffic lighta couple of traffic lights on a polea traffic light with a green light on ita green traffic light sitting next to a street,a green traffic light sitting on the side of a roada green traffic light sitting next to a traffic lighta traffic light with a green light on ita green traffic light sitting next to a streeta green traffic light sitting on the side of a street


,


,,,,
0.1659,0.1445,0.1425,0.1020,0.0936
,,,,
0.0873,0.0846,0.0838,0.0364,0.0238
,,,,
0.0226,0.0124,0.0004,0.0000,0.0000
,,,,
0.0000,0.0000,0.0000,0.0000,0.0000
,,,,
0.0000,0.0000,0.0000,0.0000,0.0000
,,,,


In [36]:
# finetuned 1.0 with closest
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a blue truck parked on top of a lush green fielda blue truck parked on top of a grass covered fielda blue truck parked on top of a grass fielda blue and blue truck parked in the grassa blue and blue truck parked in a field,a blue truck is parked in the grassa blue truck is parked on the grassa blue truck is parked in a fielda blue and blue truck parked in a fielda blue truck parked in a field with people


,


,,,,
0.2190,0.1348,0.1037,0.0992,0.0946
,,,,
0.0723,0.0711,0.0693,0.0685,0.0674


,
normal,contrastive
a black cat drinking water from a toileta black cat sitting on top of a toilet seata cat sitting on top of a toilet seata black cat sitting on top of a white toilet seata black cat sitting on top of a toilet,a black cat drinking water from a toileta black cat sitting on top of a toilet seata black cat standing on top of a toilet seata black cat sitting on top of a white toilet seata black cat drinking water from a toilet bowl


,


...

,


,,,,
0.1385,0.1258,0.1196,0.1114,0.1017
,,,,
0.1013,0.0960,0.0942,0.0661,0.0453


,
normal,contrastive
a traffic light sitting on the side of a roada green traffic light sitting on the side of a roada traffic light sitting on the side of a streeta green traffic light sitting next to a traffic lighta traffic light sitting on the side of a road next to a traffic light,a green traffic light sitting on the side of a roada green traffic light sitting next to a traffic lighta green traffic light sitting on the side of a streeta green traffic light sitting next to a street lighta green traffic light sitting next to a green light


,


,,,,
0.1623,0.1379,0.1228,0.1096,0.0960
,,,,
0.0960,0.0931,0.0615,0.0608,0.0599


In [42]:
# finetuned 1.0 with closest
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a red and white plane flying in the skya red and white airplane flying in the skya red and white plane flying in the aira red and white plane is flying in the skya red and white plane flying in a blue sky,a red and white plane flying in the skya red and white airplane flying in the skya red and white plane flying in the aira red and white plane is flying in the skya red and white plane flying in a blue sky


,


,,,,
0.1435,0.1275,0.1160,0.1153,0.1073
,,,,
0.0907,0.0902,0.0740,0.0724,0.0630


,
normal,contrastive
a city street filled with lots of tall buildingsa city street filled with lots of traffica city street filled with traffic and tall buildingsa city street filled with lots of traffic and tall buildingsa city street filled with lots of cars,a city street filled with lots of tall buildingsa city street filled with lots of traffica city street filled with traffic and tall buildingsa city street filled with lots of traffic and tall buildingsa city street filled with lots of cars


,


...

,


,,,,
0.1649,0.1472,0.1306,0.1081,0.1071
,,,,
0.0974,0.0697,0.0617,0.0606,0.0527


,
normal,contrastive
a close up of a street sign with a sky backgrounda close up of a street sign on a polea close up of a street sign with trees in the backgrounda close up of a vandalized street signa close up of a street sign near a tree,a close up of a street sign with a sky backgrounda close up of a street sign on a polea close up of a street sign with trees in the backgrounda close up of a street sign near a treea close up of a street sign with trees in background


,


,,,,
0.1910,0.1542,0.1062,0.1050,0.1037
,,,,
0.1024,0.0746,0.0576,0.0547,0.0506


In [43]:
# finetuned 1.0 with closest
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a herd of sheep standing on top of a dry grass fielda herd of sheep are standing in a fielda herd of sheep standing on top of a dirt fielda herd of sheep standing in a fielda herd of sheep standing on top of a grass covered field,a herd of sheep are standing in a fielda herd of sheep standing on top of a dry grass fielda herd of sheep standing next to each othera herd of sheep standing on top of a dirt fielda herd of sheep standing next to each other on a field


,


,,,,
0.1185,0.1182,0.1159,0.1097,0.1007
,,,,
0.0982,0.0932,0.0878,0.0815,0.0763


,
normal,contrastive
a double decker bus driving down a streeta double decker bus driving down the streeta double decker bus driving down a roada white double decker bus driving down a streeta double decker bus driving down a street next to cars,a double decker bus driving down a streeta double decker bus driving down the streeta double decker bus driving down a roada white double decker bus driving down a streeta double decker bus driving down a street next to cars


,


...

,


,,,,
0.1436,0.1354,0.1324,0.1034,0.1022
,,,,
0.0882,0.0807,0.0773,0.0721,0.0646


,
normal,contrastive
a stop sign with graffiti written on ita stop sign that has been vandalized with graffitia stop sign that has been written on ita stop sign that has been spray painted on ita stop sign with graffiti on it on a street,a red stop sign sitting on the side of a roada stop sign that has been vandalized with graffitia red stop sign sitting on the side of a streeta red stop sign sitting on the side of the roada stop sign that has been written on it


,


,,,,
0.1352,0.1344,0.1199,0.1185,0.1179
,,,,
0.0872,0.0870,0.0867,0.0829,0.0303


In [37]:
# finetuned 5.0 with closest
display_em_model(batch, k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a group of people standing around a herd of sheepa group of people standing next to a herd of sheepa group of people standing around a herd of goatsa group of people standing next to a group of sheepa group of people standing next to a herd of goats,a woman and a baby are standing in a fielda woman and a child are standing in a fielda woman and a baby are standing next to a herd of sheepa woman and a baby are standing next to a group of sheepa woman and a baby are standing next to a herd of goats


,


,,,,
0.1059,0.1036,0.1025,0.1019,0.0997
,,,,
0.0988,0.0982,0.0970,0.0968,0.0955


,
normal,contrastive
a kitchen with a stove top oven and a stovea kitchen with a stove top oven and stovea kitchen with a stove and a stovea kitchen with a white refrigerator and a stovea kitchen with a stove a stove and a microwave,a kitchen with a stove top oven and a stovethis is a kitchen with a stove top ovena kitchen with a stove a stove and a microwavea kitchen with a stove top oven and a microwavea kitchen with a stove top oven and a stove top oven


,


...

,


,,,,
0.1083,0.1080,0.1070,0.1050,0.1040
,,,,
0.0994,0.0953,0.0952,0.0947,0.0831


,
normal,contrastive
a close up of a banana on a tablea close up of a banana on a toileta close up of a banana on a platea close up of a plate of food on a tablea close up of a plate of food on a bench,a banana sitting on top of a yellow toileta banana sitting on top of a yellow fire hydranta banana sitting on top of a pile of bananasa banana sitting on top of a pile of dirta banana sitting on top of a table next to a fire hydrant


,


,,,,
0.1439,0.1328,0.1170,0.1065,0.0969
,,,,
0.0960,0.0945,0.0800,0.0700,0.0623


In [39]:
# finetuned 5.0 with closest
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a blue truck is parked on the grassa blue truck is parked in a fielda blue truck is parked in the grassa blue truck parked in a field next to peoplea blue truck parked in a field with people in the background,a blue truck is parked in a fielda blue truck is parked in the grassa man sitting in the back of a blue trucka man sitting in the back of a blue pickup trucka man sitting in the back of a truck


,


,,,,
0.1181,0.1086,0.1039,0.1021,0.0987
,,,,
0.0957,0.0940,0.0936,0.0931,0.0922


,
normal,contrastive
a cat that is sitting on a toileta cat is sitting on the toilet seata cat sitting on top of a toilet in a bathrooma cat sitting on a toilet in a bathrooma cat sitting on top of a toilet seat,a black cat sitting on top of a toileta black cat is sitting on a toileta black cat is sitting on the toileta black and white cat sitting on a toileta black cat sitting on top of a toilet seat


,


...

,


,,,,
0.1164,0.1071,0.1030,0.1024,0.1019
,,,,
0.0985,0.0979,0.0962,0.0943,0.0824


,
normal,contrastive
a couple of traffic lights on a polea couple of traffic lights on a city streeta couple of traffic lights that are on a polea street scene with focus on a traffic lighta street scene with a traffic light and a street sign,a close up of a traffic light on a city streeta close up of a traffic light on a streeta close up of a traffic light on a polea close up of a street sign on a city streeta close up of a street sign on a pole


,


,,,,
0.1136,0.1056,0.1052,0.1048,0.1010
,,,,
0.1008,0.1007,0.0899,0.0893,0.0889


In [67]:
# cbs, no EM training
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(batch, k=10, neighbor_types=['closest', 'random'], choose_distractor=True, distractor_selection_method='min')

,
normal,contrastive
a group of people standing next to each othera group of people standing around a herd of animalsa group of people standing around a herd of sheepa group of people standing next to a herd of sheepa group of people standing next to a herd of animals,a group of people standing next to each othera group of people standing around a herd of animalsa group of people standing around a herd of sheepa group of people standing next to a herd of sheepa group of people standing next to a herd of animals


,


,,,,
0.0573,0.0550,0.0544,0.0538,0.0527
,,,,
0.0525,0.0521,0.0520,0.0518,0.0505
,,,,
0.0498,0.0483,0.0481,0.0480,0.0476
,,,,
0.0475,0.0456,0.0453,0.0446,0.0432


,
normal,contrastive
a kitchen with a stove and a sinka kitchen with a stove a sink and a stovea kitchen with a stove and a stovea kitchen that has a stove and a sinka kitchen that has a stove and a stove in it,a kitchen with a stove and a sinka kitchen with a stove a sink and a stovea kitchen with a stove and a stovea kitchen that has a stove and a sinka kitchen that has a stove and a stove in it


,


...

,


,,,,
0.0580,0.0551,0.0526,0.0517,0.0515
,,,,
0.0512,0.0511,0.0509,0.0508,0.0504
,,,,
0.0501,0.0500,0.0499,0.0497,0.0497
,,,,
0.0495,0.0481,0.0469,0.0447,0.0381


,
normal,contrastive
a yellow frisbee sitting on top of a greena yellow frisbee sitting on top of a green fielda yellow banana sitting on top of a greena yellow frisbee sitting on top of a yellowa yellow banana sitting on top of a yellow,a yellow frisbee sitting on top of a greena yellow frisbee sitting on top of a green fielda yellow banana sitting on top of a greena yellow frisbee sitting on top of a yellowa yellow banana sitting on top of a yellow


,


,,,,
0.0645,0.0609,0.0603,0.0569,0.0565
,,,,
0.0527,0.0515,0.0514,0.0505,0.0497
,,,,
0.0490,0.0473,0.0470,0.0468,0.0459
,,,,
0.0450,0.0443,0.0436,0.0405,0.0359


In [43]:
# cbs, no EM training
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(batch, k=10, neighbor_types=['closest', 'random'], choose_distractor=True, distractor_selection_method='min')

,
normal,contrastive
a group of people standing around a herd of sheepa group of people standing on top of a lush green fielda group of people standing next to a herd of sheepa group of people standing on top of a fielda group of people standing on top of a field with sheep,a woman is herding a herd of sheepa woman is herding a herd of goatsa woman is herding a group of sheepa woman is herding a herd of cattlea woman is herding a herd of sheep in a field


,


,,,,
0.2650,0.1444,0.1320,0.1139,0.0712
,,,,
0.0651,0.0604,0.0409,0.0334,0.0316
,,,,
0.0103,0.0085,0.0060,0.0052,0.0035
,,,,
0.0033,0.0028,0.0012,0.0006,0.0005


,
normal,contrastive
a kitchen with a stove and a stovea kitchen that has a stove and a stovea kitchen with a stove and a sinka kitchen with a stove and a stove top ovena kitchen that has a stove and a stove in it,a kitchen with a stove and a stovea kitchen that has a stove and a stovea kitchen with a stove and a sinka kitchen with a stove and a stove top ovena kitchen that has a stove and a stove in it


,


...

,


,,,,
0.1628,0.1257,0.1163,0.1152,0.0960
,,,,
0.0887,0.0857,0.0665,0.0536,0.0167
,,,,
0.0082,0.0074,0.0074,0.0073,0.0071
,,,,
0.0071,0.0071,0.0071,0.0071,0.0071


,
normal,contrastive
a banana sitting on top of a yellow fire hydranta yellow banana sitting on top of a yellow fire hydranta yellow banana sitting on top of a yellow and black fire hydranta yellow banana sitting on top of a yellow and blue fire hydranta yellow banana sitting on top of a yellow and white fire hydrant,a banana sitting on top of a yellow fire hydranta banana sitting on the ground next to a bananaa banana sitting on the ground next to a banana peela yellow banana sitting on top of a yellow and black fire hydranta yellow banana sitting on top of a yellow and blue fire hydrant


,


,,,,
0.1095,0.1049,0.0990,0.0987,0.0901
,,,,
0.0779,0.0696,0.0387,0.0362,0.0342
,,,,
0.0330,0.0308,0.0305,0.0289,0.0247
,,,,
0.0236,0.0228,0.0159,0.0156,0.0154


In [46]:
# cbs, no EM training
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest', 'random'], choose_distractor=True, distractor_selection_method='min')

,
normal,contrastive
a red and white airplane flying in the skya red and white airplane flying through a blue skya red and white plane flying through a blue skya red and white plane flying in the skya red and white jet flying through a blue sky,a red and white jet flying through a blue skya red and white airplane flying through a blue skya red and white airplane flying in the skya red and white plane flying through a blue skya red and white plane flying in the sky


,


,,,,
0.2879,0.1966,0.1729,0.1097,0.0932
,,,,
0.0767,0.0592,0.0038,0.0000,0.0000
,,,,
0.0000,0.0000,0.0000,0.0000,0.0000
,,,,
0.0000,0.0000,0.0000,0.0000,0.0000


,
normal,contrastive
a city street filled with lots of traffica city street filled with traffic next to tall buildingsa city street filled with lots of tall buildingsa city street filled with traffic next to a tall buildinga city street filled with traffic next to a building,a city street filled with lots of traffica city street filled with traffic next to tall buildingsa city street filled with lots of tall buildingsa city street filled with traffic next to a tall buildinga city street filled with traffic next to a building


,


...

,


,,,,
0.2410,0.1453,0.1164,0.1124,0.1074
,,,,
0.0972,0.0952,0.0844,0.0005,0.0001
,,,,
0.0000,0.0000,0.0000,0.0000,0.0000
,,,,
0.0000,0.0000,0.0000,0.0000,0.0000


,
normal,contrastive
a close up of a street sign on a polea close up of a street sign with a sky backgrounda close up of a street sign on a streeta close up of a street sign with a blue skya close up of a street sign with a street sign,a close up of a street sign on a polea close up of a street sign with a sky backgrounda close up of a street sign on a streeta close up of a street sign with a blue skya close up of a street sign with a street sign


,


,,,,
0.2091,0.1978,0.1459,0.1065,0.1048
,,,,
0.0810,0.0673,0.0569,0.0119,0.0052
,,,,
0.0052,0.0037,0.0011,0.0011,0.0009
,,,,
0.0008,0.0004,0.0004,0.0002,0.0000


In [90]:
# cbs, no EM training
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a green truck parked in a grassy fielda green truck is parked on the grassa green truck is parked in a fielda green truck is parked in a grassy fielda green truck parked in a field with people in the background,a blue truck is parked in the grassa blue truck parked on top of a lush green fielda blue truck parked on top of a green fielda blue truck is parked in a grassy fielda blue truck parked on top of a grass covered field


,


,,,,
0.0914,0.0955,0.1102,0.1009,0.1020
,,,,
0.1002,0.1012,0.1019,0.0963,0.1005


,
normal,contrastive
a cat drinking water out of a toileta cat drinking water from a toilet bowla black cat drinking water from a toileta cat is drinking water from a toileta black cat drinking water out of a toilet,a cat drinking water out of a toileta black cat drinking water from a toileta cat drinking water from a toilet bowla black cat drinking water out of a toileta cat drinking water from a toilet in a bathroom


,


...

,


,,,,
0.0982,0.1074,0.1101,0.0985,0.0991
,,,,
0.0940,0.0954,0.0995,0.1051,0.0928


,
normal,contrastive
a traffic light sitting on the side of a roada green traffic light sitting on the side of a roada traffic light sitting on the side of a streeta green traffic light sitting on the side of a streeta traffic light sitting on the side of the road,a green traffic light sitting on the side of a roada green traffic light sitting on the side of a streeta green traffic light with a green lighta green traffic light sitting next to a traffic lighta green traffic light sitting next to a street


,


,,,,
0.0903,0.1044,0.0987,0.1015,0.1058
,,,,
0.1034,0.0982,0.1085,0.0930,0.0963


In [59]:
# random + closest trained
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(batch, k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a group of people standing around a herd of cattlea group of people that are standing in the grassa group of people standing next to a herd of cattlea group of people that are standing around a cowa group of people standing in a field with a cow,a group of people that are standing in the grassa group of people standing around a herd of cattlea group of people that are standing around a cowa group of people standing around a cowa group of people that are standing in a field


,


,,,,
0.1019,0.1164,0.0809,0.1164,0.1008
,,,,
0.0747,0.1164,0.0818,0.1164,0.0941


,
normal,contrastive
a kitchen with a stove and a refrigeratora kitchen with a refrigerator and a refrigeratora kitchen with a refrigerator and a stovea kitchen with a white refrigerator and a white refrigeratora kitchen with a white refrigerator freezer and a refrigerator,a kitchen with a stove and a refrigeratora kitchen with a refrigerator and a refrigeratora kitchen with a refrigerator and a stovea kitchen with a white refrigerator and a white refrigeratora kitchen with a white refrigerator freezer and a refrigerator


,


...

,


,,,,
0.1249,0.1105,0.0850,0.0422,0.0582
,,,,
0.0942,0.1147,0.1296,0.1112,0.1296


,
normal,contrastive
a yellow banana sitting next to a banana peela yellow banana sitting on top of a green fielda yellow banana sitting on the ground next to a bananaa yellow banana sitting on top of a yellow fire hydranta yellow banana sitting on top of a pile of bananas,there is a banana that is on the groundthere is a banana and a banana on the grounda yellow banana sitting on top of a green fielda yellow banana sitting on top of a yellow fire hydranta yellow banana sitting on top of a pile of bananas


,


,,,,
0.1192,0.0388,0.1192,0.0804,0.1192
,,,,
0.1012,0.1192,0.1192,0.1175,0.0662


In [60]:
# random + closest trained
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a green truck is parked in the grassa green truck is parked in a fielda green truck parked in a grassy fielda green truck parked in a field next to a foresta green truck parked in the grass next to a forest,a green truck parked in a grassy fielda green truck parked in a field next to a foresta blue truck parked in a field next to a foresta green truck parked in the grass near a foresta green truck parked in the grass next to a forest


,


,,,,
0.0589,0.1682,0.1682,0.0538,0.1161
,,,,
0.0835,0.0758,0.0748,0.0875,0.1131


,
normal,contrastive
a cat that is sitting on a toileta black cat laying on top of a toileta black cat sitting on top of a toileta black cat sitting on top of a toilet seata black cat sitting on top of a white toilet,a black cat laying on top of a toileta black cat sitting on top of a toileta black cat sitting on top of a toilet seata black cat sitting on top of a white toileta black cat laying on top of a white toilet


,


...

,


,,,,
0.0453,0.0806,0.0779,0.1123,0.1375
,,,,
0.1090,0.1375,0.0804,0.0924,0.1272


,
normal,contrastive
a traffic light sitting on the side of a roada traffic light sitting on the side of a streeta traffic light sitting on the side of the roada traffic light and a traffic light on a city streeta traffic light sitting on the side of a road next to a traffic light,a traffic light sitting on the side of a roada traffic light sitting on the side of a streeta traffic light and a traffic light on a streeta traffic light and a traffic light on a city streeta traffic light sitting on the side of a road next to a traffic light


,


,,,,
0.1000,0.1008,0.1008,0.1008,0.1008
,,,,
0.1008,0.1008,0.1008,0.0936,0.1008


In [91]:
# random + closest trained
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a red and white plane flying in the skya red and white plane flying through a blue skya red and white airplane flying in the skya red and white plane flying in the aira red and white airplane flying through a blue sky,a red and white plane flying in the skya red and white plane flying through a blue skya red and white plane flying in the aira red and white airplane flying in the skya red and white plane is flying in the sky


,


,,,,
0.0957,0.1000,0.0967,0.0989,0.1061
,,,,
0.0939,0.1046,0.1005,0.0978,0.1058


,
normal,contrastive
a city street filled with lots of tall buildingsa city street filled with lots of traffica city street filled with traffic and tall buildingsa city street filled with lots of traffic and tall buildingsa city street filled with lots of traffic and buildings,a city street filled with lots of tall buildingsa city street filled with lots of traffica city street filled with traffic and tall buildingsa city street filled with lots of traffic and tall buildingsa city street filled with lots of traffic and buildings


,


...

,


,,,,
0.0955,0.0982,0.1131,0.0897,0.1048
,,,,
0.1103,0.0908,0.0939,0.0963,0.1072


,
normal,contrastive
a close up of a street sign with a sky backgrounda close up of a vandalized street signa close up of a street sign on a polea close up of a stop sign with a sky backgrounda close up of a street sign with trees in the background,a close up of a street sign with a sky backgrounda close up of a vandalized street signa close up of a street sign on a polea close up of a stop sign with a sky backgrounda close up of a street sign with trees in the background


,


,,,,
0.1168,0.0976,0.0797,0.0919,0.1040
,,,,
0.0883,0.1170,0.1130,0.0957,0.0960


In [92]:
# random + closest trained
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a herd of sheep standing on top of a dry grass fielda herd of sheep standing on top of a grass covered fielda herd of sheep standing in a fielda large herd of sheep in a fielda herd of sheep standing on top of a dirt field,a large herd of sheep in a fielda herd of sheep standing on top of a dry grass fielda large herd of sheep standing in a fielda large herd of sheep are standing in a fielda herd of sheep standing on top of a dirt field


,


,,,,
0.0883,0.1167,0.0874,0.0897,0.1004
,,,,
0.1025,0.1181,0.0961,0.0966,0.1042


,
normal,contrastive
a double decker bus driving down a streeta double decker bus driving down the streeta double decker bus driving down a city streeta double decker bus driving down a roada double decker bus is driving down the street,a double decker bus driving down a streeta double decker bus driving down the streeta double decker bus driving down a city streeta double decker bus driving down a roada double decker bus is driving down the street


,


...

,


,,,,
0.1010,0.0996,0.0962,0.0982,0.0970
,,,,
0.0992,0.0973,0.1073,0.1082,0.0961


,
normal,contrastive
a stop sign with graffiti written on ita stop sign that has graffiti on ita red stop sign with graffiti on ita stop sign with graffiti on it on a streeta stop sign with graffiti on it on a city street,a stop sign with graffiti written on ita stop sign that has graffiti on ita stop stop sign with graffiti on ita stop sign with graffiti on it on a streeta stop sign with graffiti on it on a city street


,


,,,,
0.1020,0.1010,0.1039,0.0968,0.1079
,,,,
0.0935,0.1036,0.0937,0.0955,0.1021


In [63]:
model.op

Namespace(att_feat_size=2048, att_hid_size=512, batch_size=10, beam_size=5, bleu_reward_weight=0, block_trigrams=0, cached_tokens='coco-train-idxs', cached_transformer=False, caption_model='updown', cfg='configs/updown/updown_contrastive.yml', checkpoint_path='./log_updown_contrastive_distr-10_dct-closest+random_cbs_mlp_soft-em', cider_reward_weight=1, coco_json='', contrastive_after=0, contrastive_em='soft', current_lr=0.0004, data_in_memory=False, decoding_constraint=0, diversity_lambda=0.5, drop_prob_lm=0.5, dump_images=1, dump_json=1, dump_path=0, entropy_reward_weight=0, eval_oracle=1, fc_feat_size=2048, grad_clip_mode='value', grad_clip_value=0.1, group_size=1, id='updown_contrastive_distr-10_dct-closest+random_cbs_mlp_soft-em', image_folder='', image_root='', index_serialization_root_path='data/cocobu_indices', input_att_dir='data/cocobu_att', input_box_dir='data/cocotalk_box', input_encoding_size=1000, input_fc_dir='data/cocotalk_fc', input_json='data/cocotalk.json', input_labe

In [61]:
# random + closest trained
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest'], choose_distractor=True)

,
normal,contrastive
a red and white plane flying in the skya red and white plane flying through the skya red and white airplane is flying in the skya red and white plane is flying in the skya red and white plane flying through the air,a red and white plane flying in the skya red and white plane flying through the skya red and white airplane is flying in the skya red and white plane is flying in the skya red and white plane flying through the air


,


,,,,
0.1058,0.0904,0.0801,0.1058,0.1058
,,,,
0.0906,0.1044,0.1058,0.1058,0.1058


,
normal,contrastive
a city street filled with lots of traffica city street filled with lots of tall buildingsa city street filled with lots of traffic and tall buildingsa city street filled with lots of traffic next to tall buildingsa city street filled with lots of traffic on a city street,a city street filled with lots of traffica city street filled with lots of tall buildingsa city street filled with lots of traffic and tall buildingsa city street filled with traffic and tall buildingsa city street filled with lots of traffic next to tall buildings


,


...

,


,,,,
0.0722,0.1201,0.0754,0.1201,0.0643
,,,,
0.0698,0.1201,0.1201,0.1179,0.1201


,
normal,contrastive
a close up of a street sign on a polea close up of a street sign with a sky backgrounda close up of a street sign with a street signa close up of a street sign with a sign on ita close up of a street sign with a street sign on it,a close up of a street sign on a polea close up of a street sign with a sky backgrounda close up of a street sign with a street signa close up of a street sign with a sign on ita close up of a street sign with a street sign on it


,


,,,,
0.1133,0.0578,0.0991,0.1133,0.1133
,,,,
0.1133,0.0920,0.1133,0.0712,0.1133


In [37]:
# random + closest trained
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(batch, k=10, neighbor_types=['closest', 'random'], choose_distractor=True)

,
normal,contrastive
a group of people standing next to a herd of sheepa group of people standing around a herd of sheepa group of people standing next to a herd of animalsa group of people standing next to a herd of goatsa group of people in a field with a herd of sheep,a group of people standing next to a herd of sheepa group of people standing around a herd of sheepa group of people standing next to a herd of animalsa group of people standing next to a herd of goatsa group of people in a field with a herd of sheep


,


,,,,
0.0004,0.0004,0.0009,0.0000,0.0000
,,,,
0.0003,0.0011,0.0016,0.0019,0.0000
,,,,
0.3913,0.1622,0.0003,0.0070,0.0473
,,,,
0.0007,0.0509,0.1348,0.1899,0.0090


,
normal,contrastive
a kitchen with a stove and a stovea kitchen with a stove and a sinka kitchen with a stove and a refrigeratora kitchen with a stove and a stove top ovena kitchen that has a stove and a stove,a kitchen with a stove and a stovea kitchen with a stove and a sinka kitchen with a stove and a refrigeratora kitchen with a stove and a stove top ovena kitchen that has a stove and a stove


,


...

,


,,,,
0.0030,0.0038,0.0027,0.0030,0.0027
,,,,
0.0027,0.0030,0.0027,0.0036,0.0027
,,,,
0.0049,0.1674,0.0440,0.1349,0.2565
,,,,
0.0620,0.0520,0.0601,0.1486,0.0395


,
normal,contrastive
a yellow banana sitting on top of a green fielda yellow and yellow fire hydrant sitting on a sidewalka yellow banana sitting on top of a yellow fire hydranta yellow banana sitting on top of a pile of bananasa yellow and yellow fire hydrant sitting on top of a field,a yellow banana sitting on top of a green fielda yellow and yellow fire hydrant sitting on a sidewalka yellow banana sitting on top of a yellow fire hydranta yellow banana sitting on top of a pile of bananasa yellow and yellow fire hydrant sitting on top of a field


,


,,,,
0.0441,0.0306,0.0350,0.0086,0.0463
,,,,
0.0336,0.0222,0.0220,0.0339,0.0182
,,,,
0.0560,0.0401,0.2152,0.0257,0.0098
,,,,
0.0084,0.1820,0.0497,0.0290,0.0895


In [38]:
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest', 'random'], choose_distractor=True)

,
normal,contrastive
a green truck is parked in the grassa green truck is parked in a fielda green truck parked in the grass near a foresta green truck parked in the grass next to a foresta green truck parked in the grass next to a tree,a green truck is parked in the grassa green truck is parked in a fielda green truck parked in the grass near a foresta green truck parked in the grass next to a foresta green truck parked in the grass next to a tree


,


,,,,
0.0002,0.0000,0.0000,0.0001,0.0000
,,,,
0.0000,0.0000,0.0001,0.0000,0.0000
,,,,
0.2270,0.0315,0.0116,0.1412,0.2665
,,,,
0.1549,0.1106,0.0000,0.0081,0.0481


,
normal,contrastive
a black cat laying on top of a toileta black cat laying on top of a toilet seata black cat sitting on a toilet seata black cat laying on a toilet seata black cat is sitting on the rim of a toilet,a black cat laying on top of a toileta black cat laying on top of a toilet seata black cat sitting on a toilet seata black cat laying on a toilet seata black cat is sitting on the rim of a toilet


,


...

,


,,,,
0.0051,0.0036,0.0007,0.0005,0.0001
,,,,
0.0001,0.0003,0.0001,0.0001,0.0000
,,,,
0.2172,0.0446,0.0632,0.0001,0.0419
,,,,
0.1488,0.0820,0.1472,0.1223,0.1223


,
normal,contrastive
a couple of traffic lights on a polea traffic light sitting on the side of a roada traffic light sitting on the side of a streeta couple of traffic lights on a streeta couple of traffic lights sitting on top of a pole,a traffic light sitting on the side of a roada traffic light sitting on the side of a streeta traffic light sitting next to a traffic lighta traffic light sitting next to a streeta traffic light sitting next to a street sign


,


,,,,
0.0000,0.0000,0.0000,0.0000,0.0000
,,,,
0.0000,0.0000,0.0000,0.0000,0.0001
,,,,
0.0045,0.1784,0.2399,0.0000,0.1401
,,,,
0.2305,0.0000,0.1365,0.0694,0.0006


In [39]:
# display_em_model(batch, k=20, neighbor_types=['random', 'closest'], choose_distractor=True)
display_em_model(next(loader.iters['val']), k=10, neighbor_types=['closest', 'random'], choose_distractor=True)

,
normal,contrastive
a red and white biplane flying through a blue skya red and white airplane flying through a blue skya red and white plane flying through a blue skya red and white biplane flying through the aira red and white airplane flying through the air,a red and white biplane flying through a blue skya red and white airplane flying through a blue skya red and white plane flying through a blue skya red and white biplane flying through the aira red and white airplane flying through the air


,


,,,,
0.0000,0.0000,0.0000,0.0000,0.0000
,,,,
0.0000,0.0000,0.0001,0.0000,0.0001
,,,,
0.2752,0.0000,0.0003,0.0092,0.1791
,,,,
0.0832,0.0000,0.1496,0.1404,0.1626


,
normal,contrastive
a city street filled with lots of traffica busy city street filled with lots of traffica busy city street filled with lots of carsa city street filled with lots of traffic and traffica city street filled with lots of cars and traffic,a city street filled with lots of traffica busy city street filled with lots of traffica busy city street filled with lots of carsa city street filled with lots of traffic and traffica city street filled with lots of cars and traffic


,


...

,


,,,,
0.0000,0.0000,0.0001,0.0000,0.0000
,,,,
0.0001,0.0000,0.0000,0.0000,0.0000
,,,,
0.0172,0.0005,0.2649,0.0001,0.0774
,,,,
0.1071,0.2726,0.0013,0.0943,0.1642


,
normal,contrastive
a close up of a street sign on a polea close up of a street sign with a sky backgrounda close up of a street sign with a sign on ita close up of a street sign in front of a buildinga close up of a street sign with a street sign,a close up of a street sign on a polea close up of a street sign with a sky backgrounda close up of a street sign with a sign on ita close up of a street sign in front of a buildinga close up of a street sign with a street sign


,


,,,,
0.0007,0.0005,0.0003,0.0003,0.0014
,,,,
0.0030,0.0005,0.0009,0.0019,0.0015
,,,,
0.0367,0.1394,0.2850,0.0205,0.1103
,,,,
0.1181,0.0438,0.0003,0.0007,0.2342


In [39]:
display_em_model(batch, k=5, neighbor_type='closest', choose_distractor=False)

,
normal,contrastive
a group of people standing on top of a lush green fielda group of people standing on top of a grass covered fielda group of people standing on top of a fielda group of people standing next to a herd of sheepa group of people standing next to a group of cows,a woman holding an umbrella in a fielda woman holding an umbrella and a cowa little girl holding an umbrella in a fielda little girl holding an umbrella and a cowa woman holding an umbrella and a cow in a field


,,,,
0.2000,0.2000,0.2000,0.2000,0.2000


,
normal,contrastive
a kitchen with a stove and a refrigeratora kitchen with a stove and a sinka kitchen with a refrigerator and a sinka kitchen with a refrigerator and a stovea kitchen with a stove a sink and a stove,a kitchen with a stove and a sinka kitchen with a stove and a refrigeratora kitchen with a refrigerator and a sinka kitchen with a stove a sink and a stovea kitchen with a stove a sink and a refrigerator


,,,,
0.2000,0.2000,0.2000,0.2000,0.2000


,
normal,contrastive
a little girl holding a cat in her handa little girl holding a cat in her lapa little girl holding a cat in her handsa little girl holding a cat and a cata little girl is holding a cat in her hand,a little girl holding a cat on a leasha little girl holding a cat in her handa little girl sitting on a bench with a cata little girl holding a cat in her lapa little girl holding a cat and a cat


...

,,,,
0.2000,0.2000,0.2000,0.2000,0.2000


,
normal,contrastive
a bathroom with a sink and a mirrora bathroom with a sink and a sinka bathroom with a sink and a tuba bath room with a sink and a mirrora bathroom that has a sink and a mirror,a bathroom with a tub sink and mirrora bathroom with a sink and a tuba bath room with a sink and a mirrora bath room with a sink and a sinka bath room with a sink and a tub


,,,,
0.2000,0.2000,0.2000,0.2000,0.2000


,
normal,contrastive
there is a banana peel on the grounda yellow fire hydrant sitting on the side of a roada yellow fire hydrant sitting on the grounda yellow fire hydrant sitting on the side of the roada yellow fire hydrant sitting on top of a sidewalk,there is a banana peel on the grounda banana peel sitting on top of a yellow fire hydranta banana sitting on the ground next to a bananaa banana sitting on the ground next to a fire hydranta banana sitting on the ground next to a yellow fire hydrant


,,,,
0.2000,0.2000,0.2000,0.2000,0.2000


In [37]:
model.distractor_infos

AttributeError: 'UpDownModel' object has no attribute 'distractor_infos'

> /home/dfried/miniconda3/envs/image_captioning_pytorch/lib/python3.7/site-packages/torch/nn/modules/module.py(585)__getattr__()
    583                 return modules[name]
    584         raise AttributeError("'{}' object has no attribute '{}'".format(
--> 585             type(self).__name__, name))
    586 
    587     def __setattr__(self, name, value):

ipdb> 
ipdb> q


In [51]:
model.neighbor_infos

[[{'ix': tensor(37203),
   'id': 250804,
   'file_path': 'val2014/COCO_val2014_000000250804.jpg'}],
 [{'ix': tensor(72565),
   'id': 20966,
   'file_path': 'train2014/COCO_train2014_000000020966.jpg'}],
 [{'ix': tensor(45832),
   'id': 77693,
   'file_path': 'train2014/COCO_train2014_000000077693.jpg'}],
 [{'ix': tensor(7200),
   'id': 48910,
   'file_path': 'val2014/COCO_val2014_000000048910.jpg'}],
 [{'ix': tensor(56906),
   'id': 229962,
   'file_path': 'train2014/COCO_train2014_000000229962.jpg'}],
 [{'ix': tensor(9233),
   'id': 409855,
   'file_path': 'val2014/COCO_val2014_000000409855.jpg'}],
 [{'ix': tensor(97177),
   'id': 277050,
   'file_path': 'train2014/COCO_train2014_000000277050.jpg'}],
 [{'ix': tensor(54246),
   'id': 478675,
   'file_path': 'train2014/COCO_train2014_000000478675.jpg'}],
 [{'ix': tensor(19456),
   'id': 363321,
   'file_path': 'val2014/COCO_val2014_000000363321.jpg'}],
 [{'ix': tensor(2100),
   'id': 12323,
   'file_path': 'val2014/COCO_val2014_00000001

In [85]:
model.distractor_scorer.scorer[0].weight.norm(2, -1)

tensor([0.6026, 0.6102, 0.6137, 0.5967, 0.6030, 0.6111, 0.6166, 0.6101, 0.6100,
        0.5992, 0.6018, 0.6147, 0.5993, 0.6145, 0.6118, 0.6105, 0.6007, 0.6031,
        0.6045, 0.5957, 0.6020, 0.6081, 0.6077, 0.6051, 0.5998, 1.1390, 0.6043,
        0.6162, 0.6023, 0.6040, 0.6152, 0.6034, 0.6006, 0.5940, 0.7514, 0.5928,
        0.6147, 0.6074, 0.6116, 0.6148, 0.6010, 0.6115, 0.6109, 0.6097, 0.6141,
        0.6119, 0.5952, 0.6209, 0.6084, 0.5976, 0.6089, 0.6116, 0.6126, 0.6000,
        0.6146, 0.6141, 0.6067, 0.6163, 0.6192, 0.5979, 0.5961, 0.6064, 0.6114,
        0.5971, 0.5947, 0.6004, 0.5953, 0.6098, 0.6064, 1.0470, 0.6025, 0.6158,
        0.6044, 0.6151, 0.6055, 0.5994, 0.6419, 0.6100, 0.6038, 0.6050, 0.5956,
        0.5979, 0.6115, 0.6025, 0.6198, 0.5984, 0.6649, 1.1684, 0.5947, 0.6083,
        0.6156, 0.6136, 0.6199, 0.5962, 0.6016, 0.6144, 0.6130, 0.6060, 0.6106,
        0.6102, 0.5972, 0.6144, 0.6051, 0.5952, 0.6119, 0.6029, 0.6189, 0.6045,
        0.6128, 0.6073, 0.6073, 0.5941, 

In [53]:
# this_pred_captions = []
# for sent in seq:
#     this_pred_captions.append([
#         [model.vocab.get(str(ix.item()), 'IX_{}'.format(ix.item())) for ix in sent if ix.item() != 0]
#     ])

In [54]:
from itertools import groupby

In [55]:
from IPython.core.debugger import set_trace

In [56]:
def caption_scores(fc_feats, att_feats, att_masks, seq, add_bos=True):
    if add_bos:
        seq = torch.cat([torch.zeros(seq.size(0), 1).long().to(seq), seq], 1)
    with torch.no_grad():
        scores = model(fc_feats, att_feats, seq, att_masks)
    mask = (seq[:,:-1] > 0) | (seq[:,1:] > 0)
    # TODO: does this include the EOS score?
    # seq_t: words input at each position, with 0 for pad
    # seq_t: 0, w_0, w_1, w_2, ..., w_k, 0, ...
    # mask : 1, 1  , 1  , 1  , 1  , 1  , 0, ...
    # selected_scores: w_0 + ... + w_k
#     return scores[:,:-1].gather(2, seq[:,1:].unsqueeze(2)).squeeze(2)
    selected_scores = (scores[:,:-1].gather(2, seq[:,1:].unsqueeze(2)).squeeze(2) * mask)
    return selected_scores

In [57]:
def cross_product_scores(fc_feats, att_feats, att_masks, seq, add_bos=True):
    n_captions = seq.size(0)
    n_images = fc_feats.size(0)
    assert att_feats.size(0) == n_images
    assert att_masks.size(0) == n_images
    fc_feats_t = fc_feats.unsqueeze(0).repeat_interleave(n_captions, dim=0)
    att_feats_t = att_feats.unsqueeze(0).repeat_interleave(n_captions, dim=0)
    att_masks_t = att_masks.unsqueeze(0).repeat_interleave(n_captions, dim=0)
    
    fc_feats_t = einops.rearrange(fc_feats_t, 'caps imgs d -> (caps imgs) d')
    att_feats_t = einops.rearrange(att_feats_t, 'caps imgs obj d -> (caps imgs) obj d')
    att_masks_t = einops.rearrange(att_masks_t, 'caps imgs obj -> (caps imgs) obj')
    seq_t = seq.unsqueeze(1).repeat_interleave(n_images, dim=1)
    seq_t = einops.rearrange(seq_t, 'caps imgs d -> (caps imgs) d')
    
    scores_per_timestep = caption_scores(fc_feats_t, att_feats_t, att_masks_t, seq_t, add_bos=add_bos)
    scores = scores_per_timestep.sum(1)
    scores = einops.rearrange(scores, '(caps imgs) -> caps imgs', caps=n_captions, imgs=n_images)
    return scores

In [58]:
pdb on

Automatic pdb calling has been turned ON


In [37]:
loader.reset_iterator('val')

In [61]:
data = loader.get_batch('val')

In [62]:
this_feats = data['fc_feats']
this_ixs = [d['ix'] for d in data['infos']]
this_ids = [d['id'] for d in data['infos']]
this_paths = [d['file_path'] for d in data['infos']]

In [67]:
# set_trace()
sample_n = 10
input_data = data['fc_feats'].cuda(), data['att_feats'].cuda(), data['att_masks'].cuda(), data
n_predictions = []
eval_kwargs = {
    'sample_n_method': 'bs',
    'sample_n': sample_n,
    'temperature': 0.25,
    'verbose': False,
}
eval_utils.eval_split_n(model, n_predictions, input_data=input_data, eval_kwargs=eval_kwargs)
captions_by_id = {}
log_prob_by_id = {}
seq_by_id = {}
for k, ds in groupby(n_predictions, lambda d: d['image_id']):
    ds = list(ds)
    captions_by_id[k] = [d['caption'] for d in ds]
    log_prob_by_id[k] = [d['log_prob'] for d in ds]
    seq_by_id[k] = [d['seq'] for d in ds]
this_captions = [
    captions_by_id[id_]  for id_ in this_ids
]
this_log_probs = [
    log_prob_by_id[id_] for id_ in this_ids
]
this_seq = [
    torch.stack(seq_by_id[id_], 0) for id_ in this_ids
]

In [68]:
K = 5
for i in range(10):
    caps = this_captions[i]
#     this_s0_scores = this_log_probs[i]
    seq = this_seq[i]
    neighbor_batch = get_neighbor_batch(this_feats[i].numpy(), K, include_self=True, self_ix=this_ixs[i])
    
    # num_sampled_captions x (1+K)
    cp_scores = cross_product_scores(
        neighbor_batch['fc_feats'].cuda(),
        neighbor_batch['att_feats'].cuda(), 
        neighbor_batch['att_masks'].cuda(),
        this_seq[i].cuda()
    )
    this_s0_scores = cp_scores[:,0].detach().cpu().tolist()
    
    l1_scores = cp_scores.log_softmax(1)
    s1_scores = l1_scores.log_softmax(0)
    
    this_s1_scores = s1_scores[:,0].detach().cpu().tolist()
    
    def make_strings(lps, caps):
        scored_caps = sorted(list(zip(lps, caps)), reverse=True)
        deduped_caps = [next(g) for k, g in groupby(scored_caps, lambda t: t[1])]
        cap_strings = ["{}  {:.4f}".format(cap, lp) for lp, cap in deduped_caps]
        return cap_strings
    
    display_images([this_paths[i], this_paths[i]], 
                   captions=[
                       make_strings(this_s0_scores, caps)[:10],
                       make_strings(this_s1_scores, caps)[:10],
                   ])
    display_neighbors(data['fc_feats'][i].numpy(), k=min(K, 12), num_per_row=4)
    print()

,
a red and white airplane flying in the sky -5.3846a red and white plane flying in the sky -5.6425a red and white airplane flying in the air -6.2955a red and white airplane flying through a blue sky -6.4906a red and white airplane flying in a blue sky -6.7558a red and white plane flying in a blue sky -6.8799the red and white airplane is flying in the sky -7.9610there is a red and white plane flying in the sky -8.8055an airplane flying in the sky with a red tail -12.0166an airplane is flying in the sky with a red tail -12.4322,a red and white airplane flying in a blue sky -2.2066a red and white airplane flying in the sky -2.2307there is a red and white plane flying in the sky -2.2577the red and white airplane is flying in the sky -2.2766an airplane flying in the sky with a red tail -2.2893an airplane is flying in the sky with a red tail -2.3098a red and white plane flying in a blue sky -2.3132a red and white airplane flying in the air -2.3209a red and white airplane flying through a blue sky -2.4151a red and white plane flying in the sky -2.4287


,
a city street with lots of tall buildings -7.8097a city street with a large building in the background -10.1655a city street with a tall building in the background -10.6663a city street with a large city in the background -10.7126a city street with tall buildings and a large city -11.6744a city street with cars and a tall building -12.1900a city street with tall buildings and a large building -12.2099a city street with tall buildings and a large building in the background -12.6306a city street with cars and people walking on the sidewalk -13.4368a city street with a tall building and a large city -15.2348,a city street with tall buildings and a large building in the background -1.9513a city street with a tall building and a large city -1.9794a city street with tall buildings and a large city -2.0439a city street with tall buildings and a large building -2.1185a city street with a large city in the background -2.2861a city street with a tall building in the background -2.4150a city street with a large building in the background -2.5029a city street with lots of tall buildings -2.5392a city street with cars and a tall building -2.5657a city street with cars and people walking on the sidewalk -3.1856


,
a bus with a advertisement on the side of it -9.9457a truck with a mural on the side of it -10.0262a bus with a mural on the side of it -10.1005a blue truck with a advertisement on the side of it -10.2522a blue bus with a advertisement on the side of it -10.5892a blue truck with a sign on the side of it -11.0073a bus with a large advertisement on the side of it -11.2638a blue truck with a blue and white bus -14.9023a blue truck with a blue and white bus on the side -16.5830a blue truck with a blue and white bus on the side of it -17.4545,a blue truck with a advertisement on the side of it -1.9789a blue truck with a blue and white bus on the side of it -2.0041a blue truck with a blue and white bus on the side -2.0087a blue truck with a blue and white bus -2.1525a truck with a mural on the side of it -2.2120a blue truck with a sign on the side of it -2.2145a blue bus with a advertisement on the side of it -2.3565a bus with a mural on the side of it -2.7477a bus with a large advertisement on the side of it -2.9925a bus with a advertisement on the side of it -3.0165


,
a couple of giraffe standing next to each other -5.7969a couple of giraffe standing on top of a lush green field -6.3289a couple of giraffes are standing in a field -6.6013a couple of giraffe standing next to each other on a field -6.6032two giraffes are standing in a grassy field -6.6232two giraffes standing in the grass near trees -7.4514two giraffes standing in a field of grass -8.3326two giraffes standing in a grassy field next to trees -8.4241two giraffes standing in a field with trees -8.5482two giraffes standing in a grassy field with trees -8.6169,two giraffes standing in a grassy field with trees -1.8652two giraffes standing in the grass near trees -1.9938two giraffes are standing in a grassy field -2.0416two giraffes standing in a grassy field next to trees -2.0757two giraffes standing in a field of grass -2.2234two giraffes standing in a field with trees -2.2874a couple of giraffe standing on top of a lush green field -2.4703a couple of giraffes are standing in a field -2.8429a couple of giraffe standing next to each other -2.9213a couple of giraffe standing next to each other on a field -3.0531


,
a giraffe is laying down in the dirt -5.5802the giraffe is laying down in the dirt -6.8027a giraffe laying on the ground next to a building -7.1025a giraffe laying down in the dirt near a building -7.6570a giraffe laying down in the dirt next to a building -8.2647a giraffe laying on the ground in a zoo -8.3065a giraffe laying on the ground in a dirt -8.6407a giraffe laying down in the dirt in a zoo -8.9004a giraffe laying down on the ground in a zoo -9.0528a giraffe laying down on the ground in a dirt -9.4864,a giraffe laying down in the dirt near a building -2.1925a giraffe laying down in the dirt next to a building -2.1933a giraffe laying on the ground next to a building -2.1935a giraffe laying down on the ground in a dirt -2.3129the giraffe is laying down in the dirt -2.3140a giraffe laying down on the ground in a zoo -2.3565a giraffe laying on the ground in a dirt -2.3627a giraffe laying down in the dirt in a zoo -2.3633a giraffe laying on the ground in a zoo -2.3737a giraffe is laying down in the dirt -2.3938


,
a couple of giraffe standing next to each other -6.3707a couple of giraffe standing on top of a lush green field -6.4391a couple of giraffes are standing in a field -7.0572a couple of giraffe standing next to each other on a lush green field -7.4226two giraffes standing in the grass near trees -8.1003two giraffes standing in a grassy area next to trees -8.6406two giraffes standing in a field next to trees -9.1155two giraffes standing in a field with trees -9.2221two giraffes standing in a grassy field next to trees -9.4280two giraffes standing in a field of grass and trees -9.8691,two giraffes standing in a grassy area next to trees -1.8909two giraffes standing in the grass near trees -1.9922two giraffes standing in a field of grass and trees -2.0097two giraffes standing in a field with trees -2.1959two giraffes standing in a grassy field next to trees -2.1976two giraffes standing in a field next to trees -2.2443a couple of giraffe standing on top of a lush green field -2.4730a couple of giraffe standing next to each other on a lush green field -2.6140a couple of giraffes are standing in a field -2.8668a couple of giraffe standing next to each other -3.3280


,
a fire hydrant with a hose attached to it -7.4004a fire hydrant is on the side of the road -8.1863a fire hydrant that is on the side of the road -8.7289a fire hydrant that is on the side of a road -9.0019the fire hydrant is on the side of the road -9.0924a fire hydrant is painted white and blue -10.1041a fire hydrant is painted white and black -10.1185a fire hydrant is on the sidewalk next to a car -10.4407a fire hydrant is on a sidewalk next to a car -10.7089a fire hydrant that has been painted white and blue -11.2274,a fire hydrant is on the sidewalk next to a car -1.4475a fire hydrant is on a sidewalk next to a car -1.4559a fire hydrant with a hose attached to it -1.5761a fire hydrant is painted white and black -2.6243a fire hydrant that has been painted white and blue -2.8812a fire hydrant is on the side of the road -2.9437the fire hydrant is on the side of the road -3.1696a fire hydrant that is on the side of a road -3.2031a fire hydrant that is on the side of the road -3.2978a fire hydrant is painted white and blue -3.7296


,
a red fire hydrant sitting in the middle of a sidewalk -7.0782a red fire hydrant in front of a tree -7.2148a red fire hydrant in the middle of a park -7.5111a fire hydrant in the middle of a park -7.6126a red fire hydrant in the middle of a sidewalk -7.9263a fire hydrant in the middle of a sidewalk -8.0315a fire hydrant is spraying water onto a street -8.3755a red fire hydrant is in the middle of a sidewalk -9.2053a red fire hydrant in a city street -9.6061a red fire hydrant in a park next to a tree -10.1319,a red fire hydrant in a park next to a tree -2.0400a red fire hydrant in a city street -2.0684a red fire hydrant in front of a tree -2.0851a red fire hydrant is in the middle of a sidewalk -2.0884a red fire hydrant in the middle of a park -2.0914a red fire hydrant in the middle of a sidewalk -2.1153a red fire hydrant sitting in the middle of a sidewalk -2.1219a fire hydrant in the middle of a park -2.7492a fire hydrant in the middle of a sidewalk -3.1231a fire hydrant is spraying water onto a street -3.7515


,
a yellow and blue fire hydrant sitting on the side of a road -4.9590a yellow fire hydrant sitting on the side of a road -5.1579a blue and yellow fire hydrant sitting on the side of a road -5.4978a yellow and blue fire hydrant on a sidewalk -5.8938a blue and yellow fire hydrant on a sidewalk -6.5236the fire hydrant is on the side of the road -8.2083a fire hydrant on a sidewalk next to a street -8.2193a yellow fire hydrant on a sidewalk next to a street -8.2990a fire hydrant on a sidewalk near a street -8.3000a yellow fire hydrant on a sidewalk near a street -8.9389,a blue and yellow fire hydrant sitting on the side of a road -1.7379a blue and yellow fire hydrant on a sidewalk -1.9939a yellow and blue fire hydrant sitting on the side of a road -2.1876a yellow fire hydrant sitting on the side of a road -2.2820a yellow fire hydrant on a sidewalk near a street -2.4456a yellow fire hydrant on a sidewalk next to a street -2.4725a yellow and blue fire hydrant on a sidewalk -2.5455a fire hydrant on a sidewalk next to a street -2.5564the fire hydrant is on the side of the road -2.5935a fire hydrant on a sidewalk near a street -2.6334


,
a close up of a street sign with a sky background -5.2408a street sign that is on a pole -9.8394a street sign with a sticker on it -10.0734a street sign that says UNK and UNK -11.1867a street sign with a sign on it -12.2090a street sign with a sign that reads UNK -13.1063a sign that says UNK and UNK UNK -13.5002a street sign with a sign that says UNK -13.5340a street sign with a sticker of a man on it -14.0173a sign that says UNK UNK and a street -15.6800,a sign that says UNK UNK and a street -1.8236a street sign with a sticker of a man on it -2.0566a sign that says UNK and UNK UNK -2.0826a close up of a street sign with a sky background -2.1726a street sign with a sticker on it -2.3357a street sign with a sign that says UNK -2.3788a street sign with a sign that reads UNK -2.3977a street sign that is on a pole -2.7060a street sign with a sign on it -2.7338a street sign that says UNK and UNK -2.8123
